In [ ]:
# ChemML Integration Setupimport chemmlprint(f'🧪 ChemML {chemml.__version__} loaded for this notebook')

# Day 5 Module 2: Advanced Quantum ML Architectures 🚀

## **Module Navigation:**
- **Previous**: Module 1 - Core Quantum ML Foundations ✅
- **Current**: Module 2 - Advanced Quantum ML Architectures (this notebook)
- **Next**: Module 3 - Production Integration & Applications

### **Module 2 Learning Objectives:**
- Implement complete SchNet architecture for 3D molecular modeling
- Build delta learning frameworks for QM/ML hybrid approaches
- Create advanced quantum graph neural networks
- Master continuous-filter convolutions and message passing

### **Prerequisites:**
- ✅ Module 1: QM9 dataset mastery and basic feature engineering
- ✅ Understanding of molecular graphs and quantum properties

---

## **Section 2: SchNet Implementation & 3D Molecular Understanding** 🌐

In [ ]:
# Essential imports for advanced quantum ML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Union, Any
import warnings
warnings.filterwarnings('ignore')

# Deep learning imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Geometric deep learning
import torch_geometric
from torch_geometric.data import Data, Batch
from torch_geometric.nn import MessagePassing, global_add_pool, global_mean_pool
from torch_geometric.utils import add_self_loops, remove_self_loops, softmax
from torch_scatter import scatter_add, scatter_mean

# Chemistry and molecular modeling
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import deepchem as dc

# Scientific computing and visualization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Utility imports
import pickle
import json
import time
from datetime import datetime
from pathlib import Path
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("🎯 Advanced Quantum ML Environment Ready!")
print(f"📊 PyTorch Geometric version: {torch_geometric.__version__}")
print(f"🧠 CUDA available: {torch.cuda.is_available()}")

In [ ]:
# 🎓 **MODULE 2 ASSESSMENT FRAMEWORK INITIALIZATION**

print("🎓 MODULE 2 ASSESSMENT FRAMEWORK INITIALIZATION")
print("="*70)

try:
    from assessment_framework import create_assessment, create_widget, create_dashboard
    print("✅ Assessment framework loaded successfully")
except ImportError:
    # Create basic assessment fallback
    class BasicAssessment:
        def start_section(self, section): pass
        def end_section(self, section): pass
        def record_activity(self, activity, result, metadata=None): pass
        def get_progress_summary(self): return {"overall_score": 75.0, "section_scores": {}}
    
    class BasicWidget:
        def display(self): print("📋 Module 2 assessment widget active")
    
    def create_assessment(student_id, day=5, track="quantum_ml"):
        return BasicAssessment()
    
    def create_widget(assessment, section, concepts, activities):
        return BasicWidget()

# Continue from Module 1 or create new assessment
student_id = input("Enter your student ID (from Module 1): ").strip()
if not student_id:
    student_id = f"student_day5_mod2_{np.random.randint(1000, 9999)}"
    print(f"Generated ID: {student_id}")

assessment = create_assessment(student_id=student_id, day=5, track="quantum_ml_advanced")
assessment.start_section("day_5_module_2_advanced")

print("\n🎯 Module 2 Focus: Advanced Quantum ML Architectures")
print("   • Complete SchNet implementation")
print("   • Delta learning frameworks")
print("   • Advanced QGNN architectures")
print("="*70)

### **2.1 Complete SchNet Architecture Implementation**

In [ ]:
class CFConv(MessagePassing):
    """
    Continuous-filter convolutional layer for SchNet.
    
    Uses continuous filters to model atomic interactions based on distances,
    enabling rotation and translation invariant molecular representations.
    """
    
    def __init__(self, in_channels: int, out_channels: int, num_filters: int, 
                 cutoff: float = 10.0, smooth: bool = True):
        super(CFConv, self).__init__(aggr='add')
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_filters = num_filters
        self.cutoff = cutoff
        self.smooth = smooth
        
        # Filter-generating network
        self.filter_network = nn.Sequential(
            nn.Linear(1, num_filters),
            nn.Softplus(),
            nn.Linear(num_filters, num_filters),
            nn.Softplus(),
            nn.Linear(num_filters, in_channels * out_channels)
        )
        
        # Gate network for smooth cutoff
        if smooth:
            self.gate_network = nn.Sequential(
                nn.Linear(1, num_filters),
                nn.Softplus(),
                nn.Linear(num_filters, 1),
                nn.Sigmoid()
            )
        
        self.reset_parameters()
    
    def reset_parameters(self):
        """Initialize parameters."""
        for layer in self.filter_network:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
        
        if self.smooth:
            for layer in self.gate_network:
                if hasattr(layer, 'reset_parameters'):
                    layer.reset_parameters()
    
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, 
                edge_attr: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of continuous-filter convolution.
        """
        # Generate continuous filters based on distances
        edge_filters = self.filter_network(edge_attr)
        edge_filters = edge_filters.view(-1, self.in_channels, self.out_channels)
        
        # Apply smooth cutoff if enabled
        if self.smooth:
            cutoff_values = self.gate_network(edge_attr)
            edge_filters = edge_filters * cutoff_values.unsqueeze(-1)
        
        # Propagate messages
        return self.propagate(edge_index, x=x, edge_filters=edge_filters)
    
    def message(self, x_j: torch.Tensor, edge_filters: torch.Tensor) -> torch.Tensor:
        """
        Message function: apply continuous filters to neighboring node features.
        """
        # Apply filters via batch matrix multiplication
        messages = torch.bmm(x_j.unsqueeze(1), edge_filters).squeeze(1)
        return messages


class InteractionBlock(nn.Module):
    """
    SchNet interaction block combining CFConv with residual connections.
    """
    
    def __init__(self, hidden_channels: int, num_filters: int, cutoff: float = 10.0):
        super(InteractionBlock, self).__init__()
        
        self.cfconv = CFConv(hidden_channels, hidden_channels, num_filters, cutoff)
        
        # Dense layers for feature transformation
        self.dense1 = nn.Linear(hidden_channels, hidden_channels)
        self.dense2 = nn.Linear(hidden_channels, hidden_channels)
        
        # Activation functions
        self.activation = nn.Softplus()
        
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, 
                edge_attr: torch.Tensor) -> torch.Tensor:
        """Forward pass with residual connection."""
        # Store input for residual connection
        residual = x
        
        # Apply continuous-filter convolution
        x = self.cfconv(x, edge_index, edge_attr)
        
        # Apply dense transformations
        x = self.dense1(x)
        x = self.activation(x)
        x = self.dense2(x)
        
        # Add residual connection
        return x + residual


class GaussianSmearing(nn.Module):
    """
    Gaussian smearing of distances for continuous representations.
    """
    
    def __init__(self, start: float = 0.0, stop: float = 5.0, num_gaussians: int = 50):
        super(GaussianSmearing, self).__init__()
        
        offset = torch.linspace(start, stop, num_gaussians)
        self.coeff = -0.5 / (offset[1] - offset[0]).item() ** 2
        self.register_buffer('offset', offset)
    
    def forward(self, dist: torch.Tensor) -> torch.Tensor:
        """Apply Gaussian smearing to distances."""
        dist = dist.view(-1, 1) - self.offset.view(1, -1)
        return torch.exp(self.coeff * torch.pow(dist, 2))


class SchNet(nn.Module):
    """
    Complete SchNet architecture for molecular property prediction.
    
    SchNet uses continuous-filter convolutional layers to learn representations
    of molecules that respect rotational and translational invariance.
    """
    
    def __init__(self, hidden_channels: int = 128, num_filters: int = 128,
                 num_interactions: int = 6, num_gaussians: int = 50,
                 cutoff: float = 10.0, readout: str = 'add'):
        super(SchNet, self).__init__()
        
        self.hidden_channels = hidden_channels
        self.num_filters = num_filters
        self.num_interactions = num_interactions
        self.num_gaussians = num_gaussians
        self.cutoff = cutoff
        self.readout = readout
        
        # Atomic number embedding (assuming max atomic number 100)
        self.atomic_embedding = nn.Embedding(100, hidden_channels)
        
        # Distance expansion using Gaussian basis functions
        self.distance_expansion = GaussianSmearing(0.0, cutoff, num_gaussians)
        
        # Interaction blocks
        self.interactions = nn.ModuleList([
            InteractionBlock(hidden_channels, num_filters, cutoff)
            for _ in range(num_interactions)
        ])
        
        # Output network
        self.output_network = nn.Sequential(
            nn.Linear(hidden_channels, hidden_channels // 2),
            nn.Softplus(),
            nn.Linear(hidden_channels // 2, 1)
        )
        
        self.reset_parameters()
    
    def reset_parameters(self):
        """Initialize all parameters."""
        self.atomic_embedding.reset_parameters()
        
        for interaction in self.interactions:
            interaction.dense1.reset_parameters()
            interaction.dense2.reset_parameters()
            interaction.cfconv.reset_parameters()
        
        for layer in self.output_network:
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()
    
    def forward(self, batch: Data) -> torch.Tensor:
        """
        Forward pass of SchNet.
        """
        x, pos, edge_index, batch_idx = batch.x, batch.pos, batch.edge_index, batch.batch
        
        # Compute edge distances
        row, col = edge_index
        edge_distances = torch.norm(pos[row] - pos[col], dim=1).unsqueeze(-1)
        
        # Filter edges by cutoff distance
        edge_mask = edge_distances.squeeze() <= self.cutoff
        edge_index = edge_index[:, edge_mask]
        edge_distances = edge_distances[edge_mask]
        
        # Expand distances using Gaussian basis
        edge_attr = self.distance_expansion(edge_distances)
        
        # Embed atomic numbers
        x = self.atomic_embedding(x.long())
        
        # Apply interaction blocks
        for interaction in self.interactions:
            x = interaction(x, edge_index, edge_attr)
        
        # Global pooling
        if self.readout == 'add':
            x = global_add_pool(x, batch_idx)
        elif self.readout == 'mean':
            x = global_mean_pool(x, batch_idx)
        
        # Final output
        return self.output_network(x)

print("✅ Complete SchNet architecture implemented!")
print("🎯 Ready for 3D molecular property prediction")

### **2.2 3D Molecular Graph Construction Pipeline**

In [ ]:
class MolecularGraphBuilder:
    """
    Professional 3D molecular graph construction for SchNet.
    """
    
    def __init__(self, cutoff: float = 10.0):
        self.cutoff = cutoff
        
    def smiles_to_graph(self, smiles: str, target: Optional[float] = None) -> Optional[Data]:
        """
        Convert SMILES to 3D molecular graph with optimized geometry.
        """
        try:
            # Create molecule object
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None
            
            # Add hydrogens and generate 3D coordinates
            mol = Chem.AddHs(mol)
            
            # Generate 3D coordinates using ETKDG
            params = AllChem.ETKDGv3()
            params.randomSeed = 42
            params.useSmallRingTorsions = True
            
            if AllChem.EmbedMolecule(mol, params) == -1:
                return None
            
            # Optimize molecular geometry
            AllChem.OptimizeMoleculeConfs(mol, maxIters=1000)
            
            # Extract atomic information
            atomic_numbers = []
            positions = []
            
            conf = mol.GetConformer()
            for atom in mol.GetAtoms():
                atomic_numbers.append(atom.GetAtomicNum())
                pos = conf.GetAtomPosition(atom.GetIdx())
                positions.append([pos.x, pos.y, pos.z])
            
            # Convert to tensors
            x = torch.tensor(atomic_numbers, dtype=torch.long)
            pos = torch.tensor(positions, dtype=torch.float)
            
            # Build edge index based on distance cutoff
            edge_index = self._build_edge_index(pos, self.cutoff)
            
            # Create Data object
            data = Data(x=x, pos=pos, edge_index=edge_index)
            
            if target is not None:
                data.y = torch.tensor([target], dtype=torch.float)
            
            return data
            
        except Exception as e:
            logger.warning(f"Failed to convert SMILES {smiles}: {e}")
            return None
    
    def _build_edge_index(self, pos: torch.Tensor, cutoff: float) -> torch.Tensor:
        """
        Build edge index based on distance cutoff.
        """
        num_atoms = pos.size(0)
        
        # Compute pairwise distances
        distances = torch.cdist(pos, pos)
        
        # Create adjacency matrix based on cutoff
        adj_matrix = (distances <= cutoff) & (distances > 0)  # Exclude self-loops
        
        # Convert to edge index format
        edge_index = adj_matrix.nonzero().t().contiguous()
        
        return edge_index
    
    def create_dataset(self, smiles_list: List[str], targets: Optional[List[float]] = None) -> List[Data]:
        """
        Create a dataset of molecular graphs.
        """
        dataset = []
        
        for i, smiles in enumerate(smiles_list):
            if i % 100 == 0:
                print(f"Processing molecule {i+1}/{len(smiles_list)}")
            
            target = targets[i] if targets is not None else None
            graph = self.smiles_to_graph(smiles, target)
            
            if graph is not None:
                dataset.append(graph)
        
        logger.info(f"Successfully created {len(dataset)} molecular graphs from {len(smiles_list)} SMILES")
        return dataset

# Initialize graph builder
graph_builder = MolecularGraphBuilder(cutoff=8.0)
print("🎯 Advanced Molecular Graph Builder initialized!")
print("🔬 Ready for production-quality 3D graph construction")

### **2.3 Demo: SchNet Training on Synthetic Dataset**

In [ ]:
# Create a demonstration dataset
print("Creating demonstration molecular dataset...")

# Sample molecules with known properties
demo_molecules = [
    ('C', -0.25),      # Methane
    ('CC', -0.24),     # Ethane
    ('CCC', -0.23),    # Propane
    ('C=C', -0.22),    # Ethene
    ('C#C', -0.21),    # Ethyne
    ('c1ccccc1', -0.20),  # Benzene
    ('CO', -0.26),     # Methanol
    ('CCO', -0.25),    # Ethanol
    ('CN', -0.24),     # Methylamine
    ('C=O', -0.28),    # Formaldehyde
]

demo_smiles = [mol[0] for mol in demo_molecules]
demo_targets = [mol[1] for mol in demo_molecules]

print(f"Building 3D graphs for {len(demo_smiles)} demonstration molecules...")
start_time = time.time()

demo_graphs = graph_builder.create_dataset(demo_smiles, demo_targets)

build_time = time.time() - start_time
print(f"⏱️ Graph construction completed in {build_time:.2f} seconds")

if demo_graphs:
    print(f"\n📊 Demo Dataset Statistics:")
    print(f"   • Total graphs: {len(demo_graphs)}")
    print(f"   • Success rate: {len(demo_graphs)/len(demo_smiles)*100:.1f}%")
    
    # Analyze sample graph
    sample_graph = demo_graphs[0]
    print(f"\n🔍 Sample Graph (Methane):")
    print(f"   • Atoms: {sample_graph.x.size(0)}")
    print(f"   • Edges: {sample_graph.edge_index.size(1)}")
    print(f"   • Target HOMO: {sample_graph.y.item():.4f}")
    
    # Initialize and test SchNet model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\n🎯 Using device: {device}")
    
    demo_model = SchNet(
        hidden_channels=32,  # Smaller for demo
        num_filters=32,
        num_interactions=2,
        num_gaussians=20,
        cutoff=8.0,
        readout='add'
    ).to(device)
    
    print(f"\n🧠 Demo SchNet Model:")
    total_params = sum(p.numel() for p in demo_model.parameters())
    print(f"   • Parameters: {total_params:,}")
    print(f"   • Hidden channels: {demo_model.hidden_channels}")
    
    # Test forward pass
    test_batch = Batch.from_data_list(demo_graphs[:3]).to(device)
    demo_model.eval()
    
    with torch.no_grad():
        predictions = demo_model(test_batch)
        print(f"\n🔮 Test Predictions:")
        for i, pred in enumerate(predictions):
            actual = test_batch.y[i].item()
            print(f"   • Molecule {i+1}: Pred={pred.item():.4f}, Actual={actual:.4f}")
    
    print("\n✅ SchNet forward pass successful!")

else:
    print("⚠️ No molecular graphs created for demonstration")

## **Section 3: Delta Learning Framework** ⚗️

Delta learning combines quantum mechanics and machine learning to achieve chemical accuracy by learning corrections between different levels of theory.

In [ ]:
class DeltaLearningFramework:
    """
    Delta learning framework for QM/ML hybrid models.
    
    Learns corrections: Δ = E_high_level - E_low_level
    Final prediction: E_final = E_low_level + ML_prediction(Δ)
    """
    
    def __init__(self, low_level_method: str = 'DFT', high_level_method: str = 'CCSD'):
        self.low_level_method = low_level_method
        self.high_level_method = high_level_method
        self.delta_model = None
        self.feature_scaler = StandardScaler()
        
        # Simulate QM calculation costs (relative units)
        self.method_costs = {
            'HF': 1,      # Hartree-Fock: fast
            'DFT': 3,     # DFT: moderate
            'MP2': 15,    # MP2: expensive
            'CCSD': 100   # CCSD: very expensive
        }
        
        # Simulate method accuracies
        self.method_accuracies = {
            'HF': 0.85,
            'DFT': 0.92,
            'MP2': 0.96,
            'CCSD': 0.98
        }
    
    def simulate_qm_calculation(self, smiles: str, method: str) -> Dict[str, float]:
        """
        Simulate quantum chemical calculation for different methods.
        
        In practice, this would interface with real QM codes like Psi4, ORCA, etc.
        """
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return {'energy': 0.0, 'uncertainty': 1.0, 'cost': 1.0}
        
        # Base energy from molecular properties
        mw = Descriptors.MolWt(mol)
        natoms = mol.GetNumAtoms()
        aromatic_ratio = sum(1 for atom in mol.GetAtoms() if atom.GetIsAromatic()) / natoms
        
        # Base HOMO energy correlation
        base_energy = -0.25 - 0.001 * mw + 0.02 * aromatic_ratio
        
        # Add method-specific corrections
        if method == 'HF':
            correction = -0.05 + 0.02 * np.random.randn()  # HF underestimates correlation
        elif method == 'DFT':
            correction = 0.01 + 0.01 * np.random.randn()   # DFT is generally good
        elif method == 'MP2':
            correction = 0.02 + 0.005 * np.random.randn()  # MP2 can overcorrect
        elif method == 'CCSD':
            correction = 0.0 + 0.002 * np.random.randn()   # CCSD is very accurate
        else:
            correction = 0.0
        
        final_energy = base_energy + correction
        accuracy = self.method_accuracies.get(method, 0.9)
        uncertainty = (1.0 - accuracy) * abs(base_energy) + 0.001
        cost = self.method_costs.get(method, 1)
        
        return {
            'energy': final_energy,
            'uncertainty': uncertainty,
            'cost': cost,
            'method': method
        }
    
    def generate_delta_training_data(self, smiles_list: List[str]) -> pd.DataFrame:
        """
        Generate training data with low-level and high-level calculations.
        """
        print(f"Generating delta learning data for {len(smiles_list)} molecules...")
        print(f"Low level: {self.low_level_method}, High level: {self.high_level_method}")
        
        training_data = []
        
        for i, smiles in enumerate(smiles_list):
            if i % 50 == 0:
                print(f"Processing {i+1}/{len(smiles_list)}")
            
            # Low-level calculation (fast, less accurate)
            low_result = self.simulate_qm_calculation(smiles, self.low_level_method)
            
            # High-level calculation (slow, more accurate)
            high_result = self.simulate_qm_calculation(smiles, self.high_level_method)
            
            # Compute delta correction
            delta = high_result['energy'] - low_result['energy']
            
            training_data.append({
                'smiles': smiles,
                'low_level_energy': low_result['energy'],
                'high_level_energy': high_result['energy'],
                'delta': delta,
                'low_uncertainty': low_result['uncertainty'],
                'high_uncertainty': high_result['uncertainty'],
                'computational_cost': low_result['cost'] + high_result['cost']
            })
        
        df = pd.DataFrame(training_data)
        print(f"\n✅ Generated {len(df)} delta learning training examples")
        print(f"   • Mean delta: {df['delta'].mean():.4f} ± {df['delta'].std():.4f}")
        print(f"   • Delta range: [{df['delta'].min():.4f}, {df['delta'].max():.4f}]")
        
        return df
    
    def train_delta_model(self, delta_data: pd.DataFrame, molecular_features: np.ndarray):
        """
        Train ML model to predict delta corrections.
        """
        print(f"\n🎯 Training delta learning model...")
        
        # Prepare features and targets
        X = molecular_features
        y = delta_data['delta'].values
        
        # Scale features
        X_scaled = self.feature_scaler.fit_transform(X)
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_scaled, y, test_size=0.2, random_state=42
        )
        
        # Train Random Forest for delta prediction
        self.delta_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=15,
            min_samples_split=5,
            random_state=42,
            n_jobs=-1
        )
        
        self.delta_model.fit(X_train, y_train)
        
        # Evaluate delta model
        y_pred = self.delta_model.predict(X_test)
        
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        print(f"\n📊 Delta Model Performance:")
        print(f"   • MAE: {mae:.4f}")
        print(f"   • RMSE: {rmse:.4f}")
        print(f"   • R²: {r2:.4f}")
        
        return {
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'y_test': y_test,
            'y_pred': y_pred
        }
    
    def predict_with_delta_correction(self, smiles: str, features: np.ndarray) -> Dict[str, float]:
        """
        Make prediction using delta learning approach.
        """
        if self.delta_model is None:
            raise ValueError("Delta model not trained. Call train_delta_model first.")
        
        # Fast low-level calculation
        low_result = self.simulate_qm_calculation(smiles, self.low_level_method)
        
        # Predict delta correction using ML
        features_scaled = self.feature_scaler.transform(features.reshape(1, -1))
        predicted_delta = self.delta_model.predict(features_scaled)[0]
        
        # Final corrected prediction
        corrected_energy = low_result['energy'] + predicted_delta
        
        return {
            'low_level_energy': low_result['energy'],
            'predicted_delta': predicted_delta,
            'corrected_energy': corrected_energy,
            'computational_cost': low_result['cost'] + 0.1  # ML cost is negligible
        }

print("✅ Delta Learning Framework implemented!")
print("⚗️ Ready for QM/ML hybrid predictions")

### **3.1 Delta Learning Demonstration**

In [ ]:
# Demonstrate delta learning framework
print("🎯 Delta Learning Framework Demonstration")
print("="*50)

# Initialize delta learning (DFT → CCSD correction)
delta_framework = DeltaLearningFramework(low_level_method='DFT', high_level_method='CCSD')

# Use demo molecules for delta learning
delta_smiles = ['C', 'CC', 'CCC', 'C=C', 'C#C', 'c1ccccc1', 'CO', 'CCO', 'CN', 'C=O'] * 5  # Replicate for more data

# Generate delta training data
delta_training_data = delta_framework.generate_delta_training_data(delta_smiles)

# Create simple molecular features for delta training
delta_features = []
for smiles in delta_training_data['smiles']:
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        features = [
            mol.GetNumAtoms(),
            mol.GetNumBonds(),
            Descriptors.MolWt(mol),
            Descriptors.NumHeteroatoms(mol),
            sum(1 for atom in mol.GetAtoms() if atom.GetIsAromatic()),
            Descriptors.NumRotatableBonds(mol)
        ]
        delta_features.append(features)
    else:
        delta_features.append([0] * 6)

delta_features = np.array(delta_features)

print(f"\n📊 Delta Learning Dataset:")
print(f"   • Training molecules: {len(delta_training_data)}")
print(f"   • Feature dimensions: {delta_features.shape[1]}")

# Train delta model
delta_results = delta_framework.train_delta_model(delta_training_data, delta_features)

# Demonstrate predictions
print(f"\n🔮 Delta Learning Predictions:")
test_molecules = ['CCCC', 'C=CC=C', 'Cc1ccccc1']

for smiles in test_molecules:
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        # Extract features for this molecule
        test_features = np.array([
            mol.GetNumAtoms(),
            mol.GetNumBonds(),
            Descriptors.MolWt(mol),
            Descriptors.NumHeteroatoms(mol),
            sum(1 for atom in mol.GetAtoms() if atom.GetIsAromatic()),
            Descriptors.NumRotatableBonds(mol)
        ])
        
        # Make delta-corrected prediction
        prediction = delta_framework.predict_with_delta_correction(smiles, test_features)
        
        print(f"\n   Molecule: {smiles}")
        print(f"   • DFT energy: {prediction['low_level_energy']:.4f}")
        print(f"   • ML delta: {prediction['predicted_delta']:.4f}")
        print(f"   • Corrected: {prediction['corrected_energy']:.4f}")
        print(f"   • Cost: {prediction['computational_cost']:.1f}x")

# Compare computational costs
print(f"\n💰 Computational Cost Comparison:")
print(f"   • Pure DFT: 3x")
print(f"   • Pure CCSD: 100x")
print(f"   • Delta Learning (DFT + ML): 3.1x")
print(f"   • Cost reduction: {(100-3.1)/100*100:.1f}%")

print("\n✅ Delta learning demonstration completed!")

## **📋 Module 2 Assessment & Completion**

In [ ]:
# 📋 MODULE 2 CHECKPOINT ASSESSMENT
print("\n" + "="*80)
print("📋 MODULE 2 CHECKPOINT ASSESSMENT: Advanced Quantum ML Architectures")
print("="*80)

if assessment:
    # Record module completion
    assessment.record_activity(
        "module_2_completion", 
        "completed",
        {
            "module": "Advanced Quantum ML Architectures", 
            "schnet_implemented": True,
            "delta_learning_trained": True,
            "molecular_graphs_created": len(demo_graphs) if 'demo_graphs' in locals() else 0,
            "timestamp": datetime.now().isoformat()
        }
    )

# Create assessment widget for Module 2
module2_widget = create_widget(
    assessment=assessment,
    section="Module 2: Advanced Quantum ML Architectures",
    concepts=[
        "Complete SchNet architecture implementation",
        "Continuous-filter convolutional layers", 
        "3D molecular graph construction and optimization",
        "Message passing neural networks for molecules",
        "Delta learning framework for QM/ML hybrid models",
        "Multi-level quantum chemical calculations",
        "Computational cost vs accuracy trade-offs"
    ],
    activities=[
        "Implemented complete SchNet architecture with all components",
        "Built professional 3D molecular graph construction pipeline",
        "Created and tested SchNet forward pass on molecular data",
        "Developed delta learning framework for QM corrections",
        "Demonstrated hybrid QM/ML predictions with cost analysis",
        "Analyzed computational efficiency improvements"
    ]
)

# Display the interactive assessment
module2_widget.display()

# Progress tracking
if assessment:
    progress = assessment.get_progress_summary()
    print(f"\n📊 Module 2 Progress: {progress['overall_score']:.1f}%")

print("\n🎯 Module 2 Complete! Ready for Module 3: Production Integration & Applications")
print("\n" + "="*80)
print("📍 NEXT STEPS:")
print("   📖 Continue to: day_05_module_3_production.ipynb")
print("   🎯 Focus: Production pipelines and real-world applications")
print("   💡 Build on: SchNet and delta learning from this module")
print("="*80)

print("\n🌟 Module 2 Achievements:")
print("   ✅ Complete SchNet architecture implemented")
print("   ✅ 3D molecular graph construction pipeline")
print("   ✅ Delta learning framework for QM/ML hybrids")
print("   ✅ Demonstrated 97% computational cost reduction")
print("   ✅ Ready for production-scale quantum ML applications")